## **Warm-starting RoBERTaShared for BBC XSum**

***Note***: This notebook only uses a few training, validation, and test data samples for demonstration purposes. To fine-tune an encoder-decoder model on the full training data, the user should change the training and data preprocessing parameters accordingly as highlighted by the comments.


### **Data Preprocessing**


In [1]:
!pip install datasets -qq
!pip install transformers -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 110.1 MB/s eta 0:00:00


In [2]:
import datasets
import transformers

In [3]:
from transformers import RobertaTokenizerFast

In [4]:

tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

train_data = datasets.load_dataset("xsum", split="train[:30000]")
val_data = datasets.load_dataset("xsum", split="validation[:20%]")

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71. Subsequent calls will reuse this data.


In [5]:
batch_size=4  # change to 16 for full training
encoder_max_length=512
decoder_max_length=64

In [6]:
def process_data_to_model_inputs(batch):                                                               
    # Tokenizer will automatically set [BOS] <text> [EOS]                                               
    inputs = tokenizer(batch["document"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["summary"], padding="max_length", truncation=True, max_length=decoder_max_length)
                                                                                                        
    batch["input_ids"] = inputs.input_ids                                                               
    batch["attention_mask"] = inputs.attention_mask                                                     
    batch["decoder_input_ids"] = outputs.input_ids                                                      
    batch["labels"] = outputs.input_ids.copy()                                                          
    # mask loss for padding                                                                             
    batch["labels"] = [                                                                                 
        [-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]
    ]                     
    batch["decoder_attention_mask"] = outputs.attention_mask                                                                              
                                                                                                         
    return batch  

In [7]:
# train 
train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["document", "summary"],
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

# validation
val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["document", "summary"],
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2266 [00:00<?, ? examples/s]

### **Warm-starting the Encoder-Decoder Model**

In [ ]:
from transformers import EncoderDecoderModel

In [8]:
# set encoder decoder tying to True
roberta_shared = EncoderDecoderModel.from_encoder_decoder_pretrained("roberta-base", "roberta-base", tie_encoder_decoder=True)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.encoder.layer.2.crossattention.output.dense.weight', 'roberta.encoder.layer.11.crossattention.output.LayerNorm.weight', 'roberta.encoder.layer.8.crossattention.se

In [9]:
# set special tokens
roberta_shared.config.decoder_start_token_id = tokenizer.bos_token_id                                             
roberta_shared.config.eos_token_id = tokenizer.eos_token_id

# sensible parameters for beam search
# set decoding params                               
roberta_shared.config.max_length = 64
roberta_shared.config.early_stopping = True
roberta_shared.config.no_repeat_ngram_size = 3
roberta_shared.config.length_penalty = 2.0
roberta_shared.config.num_beams = 4
roberta_shared.config.vocab_size = roberta_shared.config.encoder.vocab_size  

### **Fine-Tuning Warm-Started Encoder-Decoder Models**

The `Seq2SeqTrainer` that can be found under [examples/seq2seq/seq2seq_trainer.py](https://github.com/huggingface/transformers/blob/master/examples/seq2seq/seq2seq_trainer.py) will be used to fine-tune a warm-started encoder-decoder model.

Let's download the `Seq2SeqTrainer` code and import the module along with `TrainingArguments`.

In [10]:
# %%capture
# !rm seq2seq_trainer.py
# !wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_trainer.py
# !pip install git-python==1.0.3
# !pip install sacrebleu==1.4.12
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=578403a7f765b0a29905d02b37247bf152beafd29db606c4ebcf8ddccb745f55
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge_score


In [18]:
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from dataclasses import dataclass, field
from typing import Optional
import numpy as np

We need to add some additional parameters to make `TrainingArguments` compatible with the `Seq2SeqTrainer`. Let's just copy the `dataclass` arguments as defined in [this file](https://github.com/patrickvonplaten/transformers/blob/make_seq2seq_trainer_self_contained/examples/seq2seq/finetune_trainer.py).

In [12]:
# @dataclass
# class Seq2SeqTrainingArguments(TrainingArguments):
#     label_smoothing: Optional[float] = field(
#         default=0.0, metadata={"help": "The label smoothing epsilon to apply (if not zero)."}
#     )
#     sortish_sampler: bool = field(default=False, metadata={"help": "Whether to SortishSamler or not."})
#     predict_with_generate: bool = field(
#         default=False, metadata={"help": "Whether to use generate to calculate generative metrics (ROUGE, BLEU)."}
#     )
#     adafactor: bool = field(default=False, metadata={"help": "whether to use adafactor"})
#     encoder_layerdrop: Optional[float] = field(
#         default=None, metadata={"help": "Encoder layer dropout probability. Goes into model.config."}
#     )
#     decoder_layerdrop: Optional[float] = field(
#         default=None, metadata={"help": "Decoder layer dropout probability. Goes into model.config."}
#     )
#     # dropout: Optional[float] = field(default=None, metadata={"help": "Dropout probability. Goes into model.config."})
#     attention_dropout: Optional[float] = field(
#         default=None, metadata={"help": "Attention dropout probability. Goes into model.config."}
#     )
#     lr_scheduler: Optional[str] = field(
#         default="linear", metadata={"help": f"Which lr scheduler to use."}
#     )

Also, we need to define a function to correctly compute the ROUGE score during validation. ROUGE is a much better metric to track during training than only language modeling loss.

In [14]:
# load rouge for validation
rouge = datasets.load_metric("rouge")

<ipython-input-14-a6cdc7186b1a>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = datasets.load_metric("rouge")


In [15]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(
            predictions=pred_str, references=label_str, rouge_types=["rouge2"]
        )["rouge2"].mid

    # Extract a few results from ROUGE
    result = {key: value * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(predict != tokenizer.pad_token_id) for predict in pred
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result
    # return {
    #     "rouge2_precision": round(rouge_output.precision, 4),
    #     "rouge2_recall": round(rouge_output.recall, 4),
    #     "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    # }

Cool! Finally, we start training.

In [21]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    num_train_epochs=5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=4,
    predict_with_generate=True,
    do_train=True,
    do_eval=True,
    logging_steps=200,  
    save_steps=500,  
    eval_steps=7500, 
    warmup_steps=500,     
    weight_decay=0.1,    
    overwrite_output_dir=True,
    save_total_limit=3
    # fp16=True, 
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=roberta_shared,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)

In [ ]:
trainer.train()

Step,Training Loss


### **Evaluation**

Awesome, we finished training our dummy model. Let's now evaluated the model on the test data. We make use of the dataset's handy `.map()` function to generate a summary of each sample of the test data.

In [ ]:
import datasets
from transformers import RobertaTokenizer, EncoderDecoderModel

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = EncoderDecoderModel.from_pretrained("./checkpoint-16")
model.to("cuda")

test_data = datasets.load_dataset("xsum", split="test")

# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
test_data = test_data.select(range(16))

batch_size = 16  # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["document"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["document"])

pred_str = results["pred"]
label_str = results["summary"]

rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

print(rouge_output)


Score(precision=0.013238526341423976, recall=0.024268890670303207, fmeasure=0.01710138826716159)


The fully trained *RoBERTaShared* model is uploaded to the 🤗model hub under [patrickvonplaten/roberta_shared_bbc_xsum](https://huggingface.co/patrickvonplaten/roberta_shared_bbc_xsum). 

The model achieves a ROUGE-2 score of **16.89**, which is a bit worse than reported in the paper. Training the model for a bit longer will most likely close this performance gap.

For some summarization examples, the reader is advised to use the online inference API of the model, [here](https://huggingface.co/patrickvonplaten/roberta_shared_bbc_xsum).